In [ ]:
import pandas as pd
import requests
from io import BytesIO
from rapidfuzz import process, fuzz
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')


In [2]:
availableDatasets_2011 = pd.read_csv('datasets_lsoa_2011.csv')
availableDatasets_2021 = pd.read_csv('datasets_lsoa_2021.csv')

def browseDatasets(searchStrings, availableDatasets):
    # Define a function to check fuzzy match
    def fuzzy_match(name, choices, threshold=98):
        match, score, _ = process.extractOne(
            name, choices, scorer=fuzz.token_set_ratio
        )
        return match if score >= threshold else None

    # Apply fuzzy matching
    availableDatasets["matched_interest"] = availableDatasets["dataset_name"].apply(
        lambda x: fuzzy_match(x, searchStrings)
    )

    # Filter only rows with a match
    matched_datasets = availableDatasets.dropna(subset=["matched_interest"])
    return matched_datasets

### Browse 2011 Census Datasets available at LSOA Level

In [3]:
# Only need LSOAs in England and Wales
availableDatasets_2011 = availableDatasets_2011[availableDatasets_2011['dataset_code'].str.endswith('EW')]

In [4]:
availableDatasets_2021

,dataset_id,dataset_name,dataset_code
0,NM_2254_1,Ability to speak Welsh by employment history,RM154
1,NM_2249_1,Ability to speak Welsh by industry,RM149
2,NM_2250_1,Ability to speak Welsh by national identity by...,RM150
3,NM_2251_1,Ability to speak Welsh by NS-SEC,RM151
4,NM_2252_1,Ability to speak Welsh by occupation,RM152
...,...,...,...
170,NM_2260_1,Welsh language skills by country of birth,RM160
171,NM_2261_1,Welsh language skills by sex by age,RM161
172,NM_2262_1,Welsh language skills by tenure - Household Re...,RM162
173,NM_2248_1,Year last worked by age,RM148


In [5]:
availableDatasets_2011

,dataset_id,dataset_name,dataset_code
0,NM_606_1,Adults not in employment and dependent childre...,KS106EW
1,NM_145_1,Age structure,KS102EW
2,NM_621_1,Car or van availability,KS404EW
4,NM_622_1,Communal establishment residents,KS405EW
6,NM_611_1,Country of birth,KS204EW
...,...,...,...
402,NM_1360_1,Passports held (detailed) (Out of term-time po...,OT212EW
403,NM_157_1,Population density (Out of term-time population),OT102EW
404,NM_1361_1,Provision of unpaid care (Out of term-time pop...,OT301EW
405,NM_1359_1,Religion (Out of term-time population),OT210EW


Find dataset with names that fuzzy matches to covariates of interest.

In [6]:
# Example covariates of interest
interests = ['Hours worked', 'NS-SeC', 'Country of birth', 
            'Health', 'Disability', 'Distance travelled to work', 'Migrant']

datasets_2011 = browseDatasets(interests, availableDatasets_2011)
datasets_2011

,dataset_id,dataset_name,dataset_code,matched_interest
6,NM_611_1,Country of birth,KS204EW,Country of birth
15,NM_617_1,Health and provision of unpaid care,KS301EW,Health
17,NM_625_1,Hours worked,KS604EW,Hours worked
35,NM_628_1,NS-SeC by sex,KS611EW to KS613EW,NS-SeC
70,NM_524_1,Country of birth (detailed),QS203EW,Country of birth
72,NM_153_1,Distance travelled to work,QS702EW,Distance travelled to work
89,NM_559_1,Hours worked,QS604EW,Hours worked
118,NM_565_1,NS-SeC of Household Reference Person (HRP) - H...,QS610EW,NS-SeC
122,NM_563_1,NS-SeC of Household Reference Person - People ...,QS608EW,NS-SeC
123,NM_564_1,NS-SeC of Household Reference Person - People,QS609EW,NS-SeC


In [7]:
availableDatasets_2021 = browseDatasets(interests, availableDatasets_2021)
availableDatasets_2021

,dataset_id,dataset_name,dataset_code,matched_interest
22,NM_2111_1,Country of birth by age,RM011,Country of birth
23,NM_2024_1,Country of birth,TS004,Country of birth
25,NM_2257_1,Disability by ability to speak Welsh by age,RM157,Disability
26,NM_2170_1,Disability by NS-SEC by age,RM070,Disability
27,NM_2172_1,Disability by religion by sex,RM072,Disability
28,NM_2056_1,Disability,TS038,Disability
29,NM_2114_1,Distance travelled to work by age,RM014,Distance travelled to work
30,NM_2116_1,Distance travelled to work by occupation,RM016,Distance travelled to work
31,NM_2075_1,Distance travelled to work,TS058,Distance travelled to work
64,NM_2156_1,Hours worked by sex by age,RM056,Hours worked


#### Manually select datasets and populate this list using their codes.

In [8]:
# Datasets of interest
dataset_ids_2011 = ['NM_611_1', 'NM_617_1', 'NM_625_1', 'NM_153_1', 'NM_559_1', 'NM_562_1', 'NM_933_1']

# View and confirm needed datasets
availableDatasets_2011[
    availableDatasets_2011['dataset_id'].isin(dataset_ids_2011)
]

,dataset_id,dataset_name,dataset_code,matched_interest
6,NM_611_1,Country of birth,KS204EW,Country of birth
15,NM_617_1,Health and provision of unpaid care,KS301EW,Health
17,NM_625_1,Hours worked,KS604EW,Hours worked
72,NM_153_1,Distance travelled to work,QS702EW,Distance travelled to work
89,NM_559_1,Hours worked,QS604EW,Hours worked
124,NM_562_1,NS-SeC,QS607EW,NS-SeC
373,NM_933_1,Country of birth,WD203EW,Country of birth


In [9]:
# Datasets of interest
dataset_ids_2021 = ['NM_2024_1', 'NM_2056_1', 'NM_2075_1', 'NM_2076_1', 'NM_2039_1', 'NM_2079_1']

# View and confirm needed datasets
availableDatasets_2021[
    availableDatasets_2021['dataset_id'].isin(dataset_ids_2021)
]

,dataset_id,dataset_name,dataset_code,matched_interest
23,NM_2024_1,Country of birth,TS004,Country of birth
28,NM_2056_1,Disability,TS038,Disability
31,NM_2075_1,Distance travelled to work,TS058,Distance travelled to work
65,NM_2076_1,Hours worked,TS059,Hours worked
88,NM_2039_1,Migrant Indicator,TS019,Migrant
108,NM_2079_1,NS-SeC,TS062,NS-SeC


In [ ]:
import os
import re
import json
import logging
import requests
import pandas as pd
from io import BytesIO

class NOMISDatasetManager:
    def __init__(self, mapping_file="column_names_mapping.json", default_output="merged_output.csv"):
        self.mapping_file = mapping_file
        self.default_output = default_output

    import requests

    def _build_url(self, dataset_id: str, year: int) -> str:
        """Build the NOMIS API URL by checking API metadata for percent and rural/urban."""
        assert year in [2011, 2021], "Year must be 2011 or 2021"

        # ----------------------------
        # 1. Check if Percent is available
        # ----------------------------
        has_percent = False
        try:
            url_measures = f"https://www.nomisweb.co.uk/api/v01/dataset/{dataset_id}.def.sdmx.json?measures"
            resp_meas = requests.get(url_measures, timeout=10)
            resp_meas.raise_for_status()
            data_meas = resp_meas.json()

            codelists = data_meas.get("structure", {}).get("codelists", {}).get("codelist", [])
            if codelists:
                codes = codelists[0].get("code", [])
                has_percent = any(
                    any(ann.get("annotationtext") == "Percent"
                        for ann in code.get("annotations", {}).get("annotation", []))
                    for code in codes
                )
        except Exception as e:
            print(f"[WARN] Could not check measures for {dataset_id}: {e}")

        measure = "20301" if has_percent else "20100"

        # ----------------------------
        # 2. Check if RURAL_URBAN is a dimension
        # ----------------------------
        has_rural_urban = False
        try:
            url_def = f"https://www.nomisweb.co.uk/api/v01/dataset/{dataset_id}.def.sdmx.json"
            resp_def = requests.get(url_def, timeout=10)
            resp_def.raise_for_status()
            data_def = resp_def.json()

            dimensions = (
                data_def.get("structure", {})
                    .get("keyfamilies", {})
                    .get("keyfamily", [])[0]
                    .get("components", {})
                    .get("dimension", [])
            )
            has_rural_urban = any(dim.get("conceptref") == "RURAL_URBAN" for dim in dimensions)
        except Exception as e:
            print(f"[WARN] Could not check dimensions for {dataset_id}: {e}")

        # ----------------------------
        # 3. Geography differs by year
        # ----------------------------
        geography = "2092957703TYPE298" if year == 2011 else "2092957703TYPE151"
        ru_filter = "&rural_urban=total" if has_rural_urban else ""

        # ----------------------------
        # 4. Final URL
        # ----------------------------
        url = (
            f"https://www.nomisweb.co.uk/api/v01/dataset/"
            f"{dataset_id}.bulk.csv?time=latest&measures={measure}&geography={geography}{ru_filter}"
        )

        return url


    def getDatasetById(self, dataset_id: str, year: int) -> pd.DataFrame | None:
        """
        Try fetching dataset with pct=True first. If it fails, retry with pct=False.
        Returns a DataFrame on success, or None on failure.
        """
        # for pct in [True, False]:
        url = self._build_url(dataset_id, year)
        try:
            logging.info(f"Fetching dataset '{dataset_id}' from {url}")

            response = requests.get(url, timeout=30)
            response.raise_for_status()

            df = pd.read_csv(BytesIO(response.content))
            logging.info(
                f"Dataset '{dataset_id}' loaded successfully "
                f"with {len(df)} rows."
            )
            return df
        except requests.exceptions.RequestException as e:
            logging.error(f"Request failed for dataset '{dataset_id}': {e}")
        except pd.errors.ParserError as e:
            logging.error(f"Failed to parse CSV for dataset '{dataset_id}': {e}")
        except Exception as e:
            logging.error(f"Unexpected error for dataset '{dataset_id}': {e}")

        logging.error(f"Unable to download dataset '{dataset_id}'.")
        return None

    def cleanDataframe(self, df, dataset_id):
        # Drop constant columns
        df = df.loc[:, df.nunique(dropna=False) > 1]

        # Clean column names
        mapping = {}
        new_cols = {}
        for col in df.columns:
            cleaned = col.split(";")[0]                # remove after ';'
            cleaned = re.sub(r'[^0-9a-zA-Z]+', ' ', cleaned)  # keep alnum, replace rest with space
            cleaned = ''.join(word.capitalize() for word in cleaned.split())  # PascalCase
            if cleaned and cleaned.lower() not in ["geography", "geographycode"]:
                cleaned = f"{dataset_id}_{cleaned}"
            new_cols[col] = cleaned
            mapping[col] = cleaned

        df = df.rename(columns=new_cols)

        # Save mapping
        self._save_mapping(dataset_id, mapping)

        return df

    def _save_mapping(self, dataset_code, mapping):
        # Append or create JSON mapping file
        if os.path.exists(self.mapping_file):
            with open(self.mapping_file, "r") as f:
                try:
                    all_mappings = json.load(f)
                except json.JSONDecodeError:
                    all_mappings = {}
        else:
            all_mappings = {}

        all_mappings[dataset_code] = mapping

        with open(self.mapping_file, "w") as f:
            json.dump(all_mappings, f, indent=4)

    def mergeDatasetsByCodes(self, dataset_ids, year, file_path=None, col_names_mapping_path=None):
        output_file = file_path or self.default_output
        self.mapping_file = col_names_mapping_path or self.mapping_file

        # Load existing merged dataframe if file exists
        if os.path.exists(output_file):
            merged_df = pd.read_csv(output_file)
            logging.info(f"Loaded existing dataframe from {output_file} with shape {merged_df.shape}")
        else:
            merged_df = None

        for id in dataset_ids:
            # Skip if already merged
            if merged_df is not None and any(col.startswith(f"{id}_") for col in merged_df.columns):
                logging.info(f"Dataset code {id} already merged. Skipping.")
                continue

            logging.info(f"Processing dataset id: {id}")
            
            
            df = self.getDatasetById(id, year)

            if df is None:
                logging.warning(f"Skipping dataset code {id} due to fetch error.")
                continue

            df = self.cleanDataframe(df, id)

            # Check required merge columns
            if not {"Geography", "GeographyCode"}.issubset(set(df.columns)):
                logging.warning(f"Dataset ID {id} missing required merge columns. Skipping.")
                continue

            # Merge
            if merged_df is None:
                merged_df = df
            else:
                merged_df = pd.merge(
                    merged_df, df,
                    on=["Geography", "GeographyCode"],
                    how='outer'
                )
   
        merged_df.to_csv(output_file, index=False)
        if merged_df is not None:
            logging.info(f"Final merged dataframe saved to {output_file} with shape {merged_df.shape}")
        else:
            logging.warning(f"Created new dataset {output_file}")

        return merged_df


In [ ]:
manager = NOMISDatasetManager()
manager.mergeDatasetsByCodes(
    dataset_ids_2011, 2011, file_path='census_data_2011_v2.csv', col_names_mapping_path='column_names_census_2011'
)


2025-09-28 01:59:48,538 - INFO - Processing dataset id: NM_611_1
2025-09-28 01:59:51,120 - INFO - Fetching dataset 'NM_611_1' from https://www.nomisweb.co.uk/api/v01/dataset/NM_611_1.bulk.csv?time=latest&measures=20301&geography=2092957703TYPE298&rural_urban=total
2025-09-28 02:01:24,757 - INFO - Dataset 'NM_611_1' loaded successfully with 34753 rows.
2025-09-28 02:01:24,798 - INFO - Processing dataset id: NM_617_1
2025-09-28 02:01:27,641 - INFO - Fetching dataset 'NM_617_1' from https://www.nomisweb.co.uk/api/v01/dataset/NM_617_1.bulk.csv?time=latest&measures=20301&geography=2092957703TYPE298&rural_urban=total
2025-09-28 02:02:57,407 - INFO - Dataset 'NM_617_1' loaded successfully with 34753 rows.
2025-09-28 02:02:57,523 - INFO - Processing dataset id: NM_625_1
2025-09-28 02:03:00,312 - INFO - Fetching dataset 'NM_625_1' from https://www.nomisweb.co.uk/api/v01/dataset/NM_625_1.bulk.csv?time=latest&measures=20301&geography=2092957703TYPE298&rural_urban=total
2025-09-28 02:04:42,368 - I

In [ ]:
manager.mergeDatasetsByCodes(
    dataset_ids_2021, 2021, columns_file_path='census_data_2021_v2.csv', col_names_mapping_path='column_names_census_2021'
)

2025-09-28 02:08:25,781 - INFO - Processing dataset id: NM_2024_1
2025-09-28 02:08:28,405 - INFO - Fetching dataset 'NM_2024_1' from https://www.nomisweb.co.uk/api/v01/dataset/NM_2024_1.bulk.csv?time=latest&measures=20301&geography=2092957703TYPE151
2025-09-28 02:09:13,183 - INFO - Dataset 'NM_2024_1' loaded successfully with 35672 rows.
2025-09-28 02:09:13,216 - INFO - Processing dataset id: NM_2056_1
2025-09-28 02:09:15,714 - INFO - Fetching dataset 'NM_2056_1' from https://www.nomisweb.co.uk/api/v01/dataset/NM_2056_1.bulk.csv?time=latest&measures=20301&geography=2092957703TYPE151
2025-09-28 02:09:38,892 - INFO - Dataset 'NM_2056_1' loaded successfully with 35672 rows.
2025-09-28 02:09:38,988 - INFO - Processing dataset id: NM_2075_1
2025-09-28 02:09:41,536 - INFO - Fetching dataset 'NM_2075_1' from https://www.nomisweb.co.uk/api/v01/dataset/NM_2075_1.bulk.csv?time=latest&measures=20301&geography=2092957703TYPE151
2025-09-28 02:10:05,271 - INFO - Dataset 'NM_2075_1' loaded successful